In [ ]:
!pip install --upgrade pip
!pip3 install tensorflow
#!pip3 install tensorflow==2.4.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [ ]:
!pip3 install tk

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
from tkinter import *
import math

In [2]:
class ShowerEnv(Env):
    def __init__(self):
        
        #Acctions up, down stay
        self.action_space = Discrete(3)
        
        #Angle array
        self.observation_space = Box(low=np.array([5]), high=np.array([14]))
        
        #Set static parameters
        self.wind_density = 1.225
        self.radious = 2
        
        #Set dynamic parameters
        self.angle = random.uniform(5.0, 14.0)
        self.wind = 10.0
        self.error = 100000
        self.power_eficiency = -0.0422*self.angle + 0.5911
        self.genPowerEuler = 0.5*self.wind_density*math.pi*pow(self.radious, 2)*pow(self.wind, 3)*self.power_eficiency
        
        #Set training time
        self.training_length = 60
        
        #Asking usr the energy
        win = Tk()
        
        e = Entry(win)
        e.pack()
        e.insert(0, "Energía")
        win.geometry("200x100")
        
        def myClick():
            global power
            power = float(e.get())
            win.quit()
            win.withdraw()
            
        myButton = Button(win, text="Iniciar", command = myClick)
        myButton.pack()

        win.mainloop()
        
        #Saving power in variable
        self.power = power
        
    def step(self, action):
        #Save the error from the previous step in a variable
        last_error = self.error
        #Reduces training time in 1 second
        self.training_length -= 1
        
        #Calculates energy
        for t in range(1, 30):
            self.power_eficiency = -0.0422*self.angle + 0.5911
            self.genPowerEuler += ((0.5*self.wind_density*math.pi*pow(self.radious, 2)*pow(self.wind, 3)*self.power_eficiency)/5 - self.genPowerEuler/5)*0.5
            #Apply action
            #0.0 - 0.1 = -0.1 (angle reduces in 0.1)
            #0.1 - 0.1 = 0.0 (angle does not change)
            #0.2 - 0.1 = 0.1 (angle increases in 0.1)
            self.angle += (action/10.0) - 0.1
            #self.error.append(abs(self.genPowerEuler - self.power))
        
        #Calculates error
        self.error = abs(self.genPowerEuler - self.power)
        
        #Calculates reward
        if self.error < last_error:
            reward = 1
        elif self.error == last_error:
            reward = -1
        else:
            reward = -10
        
        #Check if the training finished
        if self.training_length <=0:
            done = True
        else:
            done = False
            
        #Wind disturbances
        ###self.wind += random.uniform(-0.1,0.1)
        
        #placeholder for the info
        info = {}
        
        #Return step information
        return self.angle, reward, done, info
    
    def render(self):
        pass
        
    def reset(self):
        #Reset parameters
        self.angle = random.uniform(5.0, 14.0)
        self.wind = 10.0
        self.power_eficiency = -0.0422*self.angle + 0.5911
        self.genPowerEuler = 0.5*self.wind_density*math.pi*pow(self.radious, 2)*pow(self.wind, 3)*self.power_eficiency
        
        #Reset training time
        self.training_length = 60
        
        return self.angle

In [18]:
env= ShowerEnv()

C:\Users\dgarr\anaconda3\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [28]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{} Angulo:{}'.format(episode, score, n_state))


Episode:1 Score:-248 Angulo:0.3933019426711941
Episode:2 Score:-248 Angulo:15.242559518067367
Episode:3 Score:-347 Angulo:4.39683474170384
Episode:4 Score:-303 Angulo:0.3242609053761822
Episode:5 Score:-325 Angulo:-5.7586618821482585
Episode:6 Score:-325 Angulo:4.34905548297203
Episode:7 Score:-281 Angulo:9.531192143491742
Episode:8 Score:-369 Angulo:39.92428030190296
Episode:9 Score:-358 Angulo:-23.982830722234468
Episode:10 Score:-259 Angulo:1.5522779540060205


In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [20]:
states = env.observation_space.shape
actions = env.action_space.n

In [7]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape = states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [13]:
del model

In [14]:
model = build_model(states, actions)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [10]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [24]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [27]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 155s 16ms/step - reward: -3.9693
166 episodes - episode_reward: -238.253 [-435.000, -31.000] - loss: 132.180 - mae: 88.270 - mean_q: -124.944

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 153s 15ms/step - reward: -3.8667
167 episodes - episode_reward: -232.120 [-505.000, -36.000] - loss: 132.610 - mae: 88.137 - mean_q: -124.292

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 156s 16ms/step - reward: -3.4907
167 episodes - episode_reward: -209.299 [-402.000, -18.000] - loss: 128.641 - mae: 87.013 - mean_q: -121.444

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 158s 16ms/step - reward: -3.5946
166 episodes - episode_reward: -215.663 [-424.000, -28.000] - loss: 116.739 - mae: 83.835 - mean_q: -115.677

Interval 5 (40000 steps performed)
10000/10000 [===========================

In [23]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -270.000, steps: 60
Episode 2: reward: -270.000, steps: 60
Episode 3: reward: -281.000, steps: 60
Episode 4: reward: -270.000, steps: 60
Episode 5: reward: -270.000, steps: 60
Episode 6: reward: -270.000, steps: 60
Episode 7: reward: -270.000, steps: 60
Episode 8: reward: -270.000, steps: 60
Episode 9: reward: -259.000, steps: 60
Episode 10: reward: -259.000, steps: 60
Episode 11: reward: -281.000, steps: 60
Episode 12: reward: -259.000, steps: 60
Episode 13: reward: -259.000, steps: 60
Episode 14: reward: -270.000, steps: 60
Episode 15: reward: -270.000, steps: 60
Episode 16: reward: -281.000, steps: 60
Episode 17: reward: -270.000, steps: 60
Episode 18: reward: -259.000, steps: 60
Episode 19: reward: -281.000, steps: 60
Episode 20: reward: -259.000, steps: 60
Episode 21: reward: -281.000, steps: 60
Episode 22: reward: -270.000, steps: 60
Episode 23: reward: -270.000, steps: 60
Episode 24: reward: -270.000, steps: 60
Episode 25: reward: 